# Parquet konvertering-, partisjonering- og filtreringsverktøy for AIS-data

Dette verktøyet konverterer Parquet-filer til GeoParquet-format, partisjonerer på angitte kolonner og filtrerer på angitt dato og radius rundt Kristiansand sentrum. Data vises på tabell og kart.

Verktøyet er en løsningsmodell som bruker GeoPandas for følgende brukerhistorie:

"Som havnesjef i Kristiansand kommune ønsker jeg å vite hvilke skip som befant seg innen 6 km fra Kristiansand sentrum på en bestemt dato."

### Hvordan bruke dette verktøyet
1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

### Støttede filformater

- Parquet-filer med geografisk informasjon

## 2. Importer pakkene inn i verktøyet

In [6]:
import argparse
import pandas as pd
import geopandas as gpd
from shapely import wkt

from shapely import wkb
from shapely.geometry import Point, Polygon
import time
import shutil
from datetime import datetime
import os
import re
import datetime

import glob
import random

import folium
from folium import plugins
from shapely.geometry import Point, Polygon

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

from shapely.geometry import Point
import pyproj
from shapely.ops import transform
from functools import partial
import ipywidgets as widgets
from IPython.display import display


## 3. Definer funksjoner og globale variabler
Kjør cellen nedenfor for å definere funksjonene som konverterer filene:

In [7]:
# Global variabel for mappestier
_current_observer = None
global_paths = None

# Sett standardverdier for parameterne
data_dir = "./data"
num_files = 5

# Standard skiptype (foreløpig)
ship_type = 60

# Output-felt for statusmeldinger
status_output = widgets.Output()

def find_newest_folder():
    """
    Finner den nyeste mappen i en mappe basert på mappenavnet (dato) eller modifiseringstidspunkt.
    Leter etter mappenavn på formatet: hais_2024-12-24.snappy.parquet

    Args:
        folder_path (str): Stien til mappen hvor undermappene skal søkes
        use_foldername (bool): Om dato i mappenavn skal brukes (True) eller sist endret tid (False)

    Returns:
        tuple: (str: Stien til den nyeste mappen, str: Datoen som string i format 'YYYY-MM-DD',
               eller None hvis det ikke finnes mapper eller dato ikke ble funnet)
    """
    folder_path = "data/processed"
    use_foldername=True
    try:
        # Sjekk om mappen eksisterer
        if not os.path.exists(folder_path):
            print(f"Mappen {folder_path} eksisterer ikke")
            return None

        # Finn alle mapper i mappen (ikke filer)
        folders = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                  if os.path.isdir(os.path.join(folder_path, f))]

        if not folders:
            print(f"Ingen mapper funnet i {folder_path}")
            return None

        if use_foldername:
            # Finn den nyeste mappen basert på dato i mappenavnet
            newest_date = None
            newest_folder = None
            date_string = None

            for folder in folders:
                foldername = os.path.basename(folder)
                # Søk etter dato i formatet YYYY-MM-DD i mappenavnet
                date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)

                if date_match:
                    date_str = date_match.group(1)
                    try:
                        folder_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

                        if newest_date is None or folder_date > newest_date:
                            newest_date = folder_date
                            newest_folder = folder
                            date_string = date_str
                    except ValueError:
                        # Hvis datoen ikke kan tolkes, hopp over denne mappen
                        continue

            if newest_folder:
                return date_string
            else:
                print("Ingen mapper med gyldig datoformat funnet. Bruker modifiseringstidspunkt i stedet.")
                # Fall tilbake til modifiseringstidspunkt
                newest_folder = max(folders, key=os.path.getmtime)
                # Ingen dato funnet i mappenavn, så vi returnerer None som date_string
                return newest_folder, None
        else:
            # Finn den nyeste mappen basert på modifiseringstidspunkt
            newest_folder = max(folders, key=os.path.getmtime)

            # Sjekk om vi kan finne en dato i mappenavnet til den nyeste mappen
            foldername = os.path.basename(newest_folder)
            date_match = re.search(r'(\d{4}-\d{2}-\d{2})', foldername)
            date_string = date_match.group(1) if date_match else None

            return date_string

    except Exception as e:
        print(f"Feil ved søk etter nyeste mappe: {str(e)}")
        return None

def create_folders(data_folder="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_folder: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    raw_folder = os.path.join(data_folder, "raw")
    processed_folder = os.path.join(data_folder, "processed")
    source_folder = os.path.join(data_folder, "source")
    incoming_folder = os.path.join(data_folder, "incoming")

    # Opprett mapper
    os.makedirs(data_folder, exist_ok=True)
    os.makedirs(raw_folder, exist_ok=True)
    os.makedirs(processed_folder, exist_ok=True)
    os.makedirs(source_folder, exist_ok=True)
    os.makedirs(incoming_folder, exist_ok=True)

    # Returner stier for senere bruk
    return {
        "data_folder": os.path.abspath(data_folder),
        "raw_folder": os.path.abspath(raw_folder),
        "processed_folder": os.path.abspath(processed_folder),
        "source_folder": os.path.abspath(source_folder),
        "incoming_folder": os.path.abspath(incoming_folder)
    }

def convert_parquet_to_geoparquet(file_path, output_path, partition_columns=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        file_path: Sti til parquet-filen
        output_path: Sti hvor GeoParquet-filen skal lagres
        partition_columns: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    time_partitioning = False
    if partition_columns and 'date_time_utc' in partition_columns:
        partition_columns.remove('date_time_utc')
        time_partitioning = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if time_partitioning:
        df = add_time_partitioning_columns(df)
        # Sett opp partisjonering kun på 'hour'
        partition_columns = ['hour'] + (partition_columns or [])

    # Opprett GeoDataFrame
    gdf = create_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partition_columns and not all(col in gdf.columns for col in partition_columns):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        missing = [col for col in partition_columns if col not in gdf.columns]
        print(f"Manglende kolonner: {missing}")
        return False

    # Lagre med partisjonering (bare time)
    if partition_columns:
        return save_partitioned_geoparquet(gdf, output_path, partition_columns)
    else:
        gdf.to_parquet(output_path)
        return output_path

def add_time_partitioning_columns(df, time_column='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if time_column not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{time_column}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[time_column]):
        print(f"Advarsel: Kolonnen '{time_column}' er ikke en datetime-kolonne")
        return df

    df_with_time = df.copy()
    df_with_time['year'] = df_with_time[time_column].dt.year
    df_with_time['month'] = df_with_time[time_column].dt.month
    df_with_time['day'] = df_with_time[time_column].dt.day
    df_with_time['hour'] = df_with_time[time_column].dt.hour

    return df_with_time

def create_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """

    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_columns = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_columns:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def save_partitioned_geoparquet(gdf, output_path, partition_columns):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(output_path, partition_cols=partition_columns)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    convert_partitioned_files_to_geoparquet(output_path)
    return output_path

def convert_partitioned_files_to_geoparquet(root_folder):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    error_count = 0

    for root, _, files in os.walk(root_folder):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                error_count += 1

    if error_count > 0:
        print(f"Advarsel: {error_count} filer kunne ikke konverteres til GeoParquet")

def define_agder_polygon():
    """
    Definerer et forenklet polygon for Agder-kysten.

    Returns:
        Shapely Polygon som definerer Agder-kystlinjen
    """

    agder_coords = [
        (6.8, 57.8),  # Sørvest
        (10.0, 57.8),  # Sørøst
        (10.0, 59.5),  # Nordøst
        (6.8, 59.5),  # Nordvest
        (6.8, 57.8)   # Lukk polygonet
    ]
    return Polygon(agder_coords)

def filter_ais_data(root_folder, date=None, filters=None, agder_polygon=None, max_number_of_ships=10):
    """
    Filtrerer AIS-data basert på dato og geografisk område.

    Args:
        root_folder: Mappen hvor dataene er lagret
        date: Dato for filtrering
        filters: Dictionary med kolonnenavn og verdier for filtrering
        agder_polygon: Polygon som definerer Agder-kysten
        max_number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if not date:
        print("Dato er påkrevd")
        return None

    if filters is None:
        filters = {}

    # Dato-komponenter for filtrering senere
    year, month, day = date.year, date.month, date.day

    # Konverter dato til string for bruk i filnavn
    dato_str = date.strftime('%Y-%m-%d')


    # Sjekk om rot-mappen eksisterer
    if not os.path.exists(root_folder):
        print(f"Mappen {root_folder} finnes ikke")
        return None

    # Finn alle mapper i rot-mappen
    date_folders = []
    for element in os.listdir(root_folder):
        full_path = os.path.join(root_folder, element)
        if os.path.isdir(full_path):
            date_folders.append(element)

    if not date_folders:
        print(f"Ingen datamapper funnet i {root_folder}")
        return None

    # Finn alle parquet-filer rekursivt
    date_found = []

    for date_folder in date_folders:
        base_path = os.path.join(root_folder, date_folder)
        # Traverser mappestrukturen rekursivt
        for root, dirs, files in os.walk(base_path):
            # For time-partisjonering, sjekk fil-innhold istedenfor å basere på katalogstruktur
            for file in files:
                if file.endswith('.parquet'):
                    file_path = os.path.join(root, file)
                    date_found.append(file_path)

    if not date_found:
        print(f"Ingen parquet-filer funnet")
        return None

    # Les inn og filtrer dataene
    dfs = []
    for file_path in date_found:
        try:
            # Ekstraher partisjonsverdier fra stien (for time)
            time_value = None
            parts = file_path.split(os.sep)
            for part in parts:
                if part.startswith('hour='):
                    time_value = int(part.split('=')[1])

            # Les parquet-filen
            df = pd.read_parquet(file_path)

            # Legg til hour som kolonne hvis den ikke allerede finnes
            if time_value is not None and 'hour' not in df.columns:
                df['hour'] = time_value

            # Filtrer på dato
            if 'year' in df.columns and 'month' in df.columns and 'day' in df.columns:
                df = df[(df['year'] == year) & (df['month'] == month) & (df['day'] == day)]

                if df.empty:
                    # Hopp over denne filen hvis ingen rader matcher datoen
                    continue

            # Utfør ekstra filtrering basert på filtre-parameteren
            ship_df = False
            for columns, value in filters.items():
                if columns not in df.columns:
                    ship_df = True
                    break

                # For alle kolonner, filtrer på vanlig måte
                if isinstance(value, list):
                    if not df[columns].isin(value).any():
                        ship_df = True
                        break
                else:
                    if not (df[columns] == value).any():
                        ship_df = True
                        break

            if not ship_df:
                # Gjør ytterligere filtrering på dataframe-nivå
                for columns, value in filters.items():
                    if columns in df.columns:
                        if isinstance(value, list):
                            df = df[df[columns].isin(value)]
                        else:
                            df = df[df[columns] == value]

                if not df.empty:
                    dfs.append(df)

        except Exception as e:
            print(f"Feil ved lesing av {file_path}: {str(e)}")

    if not dfs:
        print("Ingen data lest inn")
        return None

    # Slå sammen alle dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Konverter til GeoDataFrame hvis koordinater finnes
    if 'longitude' in combined_df.columns and 'latitude' in combined_df.columns:
        # Opprett geometrikolonne fra longitude og latitude
        geometry = [Point(xy) for xy in zip(combined_df['longitude'], combined_df['latitude'])]
        geo_df = gpd.GeoDataFrame(combined_df, geometry=geometry, crs="EPSG:4326")

        # Filtrer basert på Agder-polygon hvis gitt
        if agder_polygon is not None:
            # Sjekk om agder_polygon er et GeoDataFrame eller et Shapely-objekt
            if isinstance(agder_polygon, gpd.GeoDataFrame):
                # Sørg for at CRS er likt
                if agder_polygon.crs != geo_df.crs:
                    agder_polygon = agder_polygon.to_crs(geo_df.crs)

                # Spatial join - dette kan ta litt tid for store datasett
                geo_df = gpd.sjoin(geo_df, agder_polygon, how="inner", predicate="within")
            else:
                # Antar at agder_polygon er et Shapely-objekt
                geo_df = geo_df[geo_df.geometry.within(agder_polygon)]

        # Sorter etter nyeste data først for hvert skip
        if 'timestamp' in geo_df.columns:
            geo_df = geo_df.sort_values(by='timestamp', ascending=False)

        # Begrens til unike skip
        if 'mmsi' in geo_df.columns or 'ship_name' in geo_df.columns:
            id_columns = 'mmsi' if 'mmsi' in geo_df.columns else 'ship_name'
            unique_ships = geo_df[id_columns].unique()

            if len(unique_ships) > max_number_of_ships:
                print(f"Begrenser til {max_number_of_ships} skip")
                unique_ships = unique_ships[:max_number_of_ships]

            geo_df = geo_df[geo_df[id_columns].isin(unique_ships)]

            # Hent første rad for hvert unikt skip for å få en kompakt liste
            geo_df = geo_df.drop_duplicates(subset=[id_columns])

        return geo_df
    else:
        print("Mangler koordinater (longitude/latitude)")
        return None

def find_ships_near_point(ship_gdf, point, buffer_distance=500):
    """
    Filtrerer skip som er innenfor en gitt avstand fra et punkt.

    Args:
        ship_gdf: GeoDataFrame med skip
        point: Tuple (longitude, latitude)
        buffer_distance: Avstand i meter

    Returns:
        GeoDataFrame med filtrerte skip
    """
    print("Finner skip i radius av Kristiansand ...")
    time.sleep(2)

    if ship_gdf is None or ship_gdf.empty:
        return None

    try:

        # Opprett punkt fra koordinater
        center_point = Point(point)

        # Projiser til et lokalt koordinatsystem som bruker meter som enheter (UTM-sone)
        # Finn UTM-sone basert på longitude
        utm_sone = int((point[0] + 180) / 6) + 1
        utm_crs = f"EPSG:326{utm_sone}"  # Nordlig halvkule

        # Transformer funksjonen for å lage buffer i meter
        project_to_utm = pyproj.Transformer.from_crs(
            "EPSG:4326", utm_crs, always_xy=True).transform
        project_to_wgs84 = pyproj.Transformer.from_crs(
            utm_crs, "EPSG:4326", always_xy=True).transform

        # Projiser punkt til UTM, lag buffer i meter, projiser tilbake til WGS84
        utm_point = transform(project_to_utm, center_point)
        buffer_utm = utm_point.buffer(buffer_distance)
        buffer_wgs84 = transform(project_to_wgs84, buffer_utm)

        # Filtrer for skip innenfor bufferen
        ships_within_buffer = ship_gdf[ship_gdf.geometry.within(buffer_wgs84)]
        print(f"Fant {len(ships_within_buffer)} skip.")

        return ships_within_buffer, buffer_wgs84

    except Exception as e:
        print(f"Feil ved buffering: {str(e)}")
        return None, None

def filter_ships_in_agder(root_folder, date, radius=None, center_point=(8.0182, 58.1599), number_of_ships=10):
    """
    Filtrerer skip i Agder-området, enten basert på et definert polygon eller en radius fra et senterpunkt.

    Args:
        root_folder: Mappen hvor de konverterte dataene er lagret
        date: Datoen å filtrere på (datetime objekt)
        radius: Radius i meter fra senterpunkt (None = bruk polygon)
        center_point: Tuple (longitude, latitude) for senterpunkt (standard: Kristiansand)
        number_of_ships: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """

    # Først hent alle skip for datoen uten geografisk filtrering
    ships_on_date = filter_ais_data(
        root_folder=root_folder,
        date=date,
        filters=None,
        agder_polygon=None,  # Ingen geografisk filter her ennå
        max_number_of_ships=100  # Hent litt flere for å kunne filtrere geografisk etterpå
    )

    if ships_on_date is None or ships_on_date.empty:
        print(f"Ingen skip funnet på datoen {date.strftime('%d.%m.%Y')}")
        return None

    # Hvis radius er oppgitt, bruk finn_skip_rundt_punkt
    if radius is not None and radius > 0:
        print(f"Søker etter skip innen {radius} meter fra Kristiansand sentrum ...")
        ships_near_point, buffer = find_ships_near_point(
            ships_on_date,
            center_point,
            buffer_distance=radius
        )

        # Begrens antall skip hvis nødvendig
        if ships_near_point is not None and len(ships_near_point) > number_of_ships:
            ships_near_point = ships_near_point.head(number_of_ships)

        if ships_near_point is None or ships_near_point.empty:
            print(f"Ingen skip funnet innenfor {radius}m fra Kristiansand sentrum på datoen {date.strftime('%d.%m.%Y')}")
            return None

        print(f"Fant {len(ships_near_point)} skip innenfor {radius}m på datoen {date.strftime('%d.%m.%Y')}")
        return ships_near_point
    else:
        print("BRUKER STD AGDER POL")
        # Bruk standard Agder-polygon
        agder_polygon = define_agder_polygon()

        # Filtrer med polygon
        ships_within_agder = ships_on_date[ships_on_date.geometry.within(agder_polygon)]

        # Begrens antall skip
        if len(ships_within_agder) > number_of_ships:
            ships_within_agder = ships_within_agder.head(number_of_ships)

        if ships_within_agder.empty:
            print(f"Ingen skip funnet innenfor Agder-område på datoen {date.strftime('%d.%m.%Y')}")
            return None

        print(f"Fant {len(ships_within_agder)} skip innenfor Agder-område på datoen {date.strftime('%d.%m.%Y')}")
        return ships_within_agder

def visualize_ships(ship_gdf, agder_polygon=None, tittel="Skip innenfor Agder"):
    """
    Visualiserer skip på et interaktivt kart med Folium.

    Args:
        ship_gdf: GeoDataFrame med skip
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)
        tittel: Tittel på kartet

    Returns:
        Folium Map objekt
    """
    if ship_gdf is None or ship_gdf.empty:
        print("Ingen skip å visualisere")
        return None

    # Beregn senterpunkt for kartet basert på skipene
    map_center = [ship_gdf.geometry.y.mean(), ship_gdf.geometry.x.mean()]

    # Opprett kartet
    m = folium.Map(
        location=map_center,
        zoom_start=10,
        tiles='OpenStreetMap',
        control_scale=True
    )

    # Legg til Agder-polygon hvis tilgjengelig
    if agder_polygon is not None:
        if isinstance(agder_polygon, gpd.GeoDataFrame):
            # Hvis det er en GeoDataFrame, konverter til GeoJSON og legg til
            folium.GeoJson(
                data=agder_polygon,
                name="Agder fylke",
                style_function=lambda x: {
                    'fillColor': '#aaaaff',
                    'color': '#0000ff',
                    'weight': 2,
                    'fillOpacity': 0.1
                }
            ).add_to(m)
        else:
            # Hvis det er et Shapely polygon, konverter koordinater og legg til
            if isinstance(agder_polygon, Polygon):
                coords = list(agder_polygon.exterior.coords)
                # Konverter til format [lat, lon] som Folium forventer
                folium_coords = [[y, x] for x, y in coords]

                folium.Polygon(
                    locations=folium_coords,
                    color='blue',
                    weight=2,
                    fill=True,
                    fill_color='#aaaaff',
                    fill_opacity=0.1,
                    name='Agder fylke'
                ).add_to(m)

    # Opprett en MarkerCluster for skipene
    marker_cluster = plugins.MarkerCluster(name="Skip").add_to(m)

    # Legg til markører for hvert skip
    for idx, row in ship_gdf.iterrows():
        # Samle informasjon for popup
        popup_info = "<table>"
        for col in ship_gdf.columns:
            if col != 'geometry' and pd.notna(row[col]):
                popup_info += f"<tr><th>{col}</th><td>{row[col]}</td></tr>"
        popup_info += "</table>"

        # Opprett popup
        popup = folium.Popup(popup_info, max_width=300)

        # Hent skipnavn
        skip_navn = row.get('ship_name', f"Skip {idx}")

        # Opprett ikon
        icon = folium.Icon(icon='ship', prefix='fa', color='blue')

        # Legg til markør
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=popup,
            tooltip=skip_navn,
            icon=icon
        ).add_to(marker_cluster)

    # Legg til fullskjermsknapp
    plugins.Fullscreen().add_to(m)

    # Legg til lagkontroll (for å slå av/på lag)
    folium.LayerControl().add_to(m)

    # Legg til tegneforklaring
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                background-color: white; padding: 10px; border-radius: 5px;
                border: 2px solid grey; font-size: 14px;">
      <p><b>Tegnforklaring</b></p>
      <p>
        <i class="fa fa-ship fa-1x" style="color:blue"></i> Skip<br>
        <span style="background-color: #aaaaff; opacity: 0.1; border: 1px solid blue;">&nbsp;&nbsp;&nbsp;&nbsp;</span> Agder fylke
      </p>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    #m.fit_bounds(m.get_bounds())

    return m

def visualize_ships_near_point(ship_gdf, punkt, buffer_avstand=500, agder_polygon=None):
    """
    Visualiserer skip innenfor en buffer rundt et gitt punkt.

    Args:
        ship_gdf: GeoDataFrame med skip
        punkt: Tuple (longitude, latitude)
        buffer_avstand: Avstand i meter
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)

    Returns:
        Folium Map objekt
    """
    # Finn skip innenfor buffer
    ships_inside_buffer, buffer_wgs84 = find_ships_near_point(ship_gdf, punkt, buffer_avstand)

    if ships_inside_buffer is None or ships_inside_buffer.empty:
        print(f"Ingen skip funnet innenfor {buffer_avstand}m fra punktet {punkt}")

        # Opprett et kart selv om ingen skip ble funnet, for å vise punktet og bufferen
        map_center = [punkt[1], punkt[0]]  # [lat, lon]
        m = folium.Map(
            location=map_center,
            zoom_start=10,
            tiles='CartoDB positron',
            control_scale=True
        )

        # Legg til flere bakgrunnskart
        folium.TileLayer('OpenStreetMap').add_to(m)
        folium.TileLayer('CartoDB dark_matter').add_to(m)
        folium.TileLayer('Stamen Terrain').add_to(m)

        # Legg til markør for senterpunkt
        folium.Marker(
            location=[punkt[1], punkt[0]],
            popup=f"Koordinater: {punkt}",
            tooltip="Senterpunkt",
            icon=folium.Icon(icon='crosshairs', prefix='fa', color='red')
        ).add_to(m)

        # Legg til sirkel for buffer
        folium.Circle(
            location=[punkt[1], punkt[0]],
            radius=buffer_avstand,
            color='red',
            fill=True,
            fill_color='#ff7777',
            fill_opacity=0.2,
            popup=f"Buffer: {buffer_avstand}m"
        ).add_to(m)

        # Legg til fullskjermsknapp og lagkontroll
        plugins.Fullscreen().add_to(m)
        folium.LayerControl().add_to(m)

        return m

    # Opprett kart
    tittel = f"Skip innenfor {buffer_avstand}m fra Kristiansand sentrum:"
    m = visualize_ships(ships_inside_buffer, agder_polygon, tittel)

    # Legg til markør for senterpunkt
    folium.Marker(
        location=[punkt[1], punkt[0]],
        popup=f"Koordinater: {punkt}",
        tooltip="Senterpunkt",
        icon=folium.Icon(icon='crosshairs', prefix='fa', color='red')
    ).add_to(m)

    # Legg til sirkel for buffer
    folium.Circle(
        location=[punkt[1], punkt[0]],
        radius=buffer_avstand,
        color='red',
        fill=True,
        fill_color='#ff7777',
        fill_opacity=0.2,
        popup=f"Buffer: {buffer_avstand}m"
    ).add_to(m)

    return m

def validate_date(date_str):
    try:
        # Prøv å konvertere strengen til en dato
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        return date
    except ValueError:
        return None

def run_vis():
    global radius_slider

    print("Visualiserer data ...")

    # Hent dato
    date_str = find_newest_folder()
    if not date_str:
        return

    # Valider datoen
    date = validate_date(date_str)

    if date is None:
        print(f"Ugyldig datoformat. Vennligst bruk formatet ÅÅÅÅ-MM-DD (f.eks. 2024-04-24)")
        return

    # Hent radius fra slider
    radius = radius_slider.value

    # Standard senterpunkt (Kristiansand)
    center_point = (8.0182, 58.1599)

    # Kall filtreringsfunksjonen med dato, radius og senterpunkt
    root_folder = "data/processed"

    try:
        # Først prøv med enkel versjon som i kode 1 (hvis mulig)
        ships_in_agder = filter_ships_in_agder(root_folder, date, radius,
                center_point, 10)
        if ships_in_agder is None or ships_in_agder.empty:
            print("Ingen skip funnet med enkel filtrering, prøver med utvidede parametre...")
            # Hvis ingen resultater, prøv med radius og senterpunkt
            ships_in_agder = filter_ships_in_agder(
                root_folder,
                date,
                radius,
                center_point,
                20  # Økt fra 10 til 20 for å tillate flere skip
            )
    except TypeError:
        # Hvis filter_ships_in_agder krever alle parametre
        print("Bruker utvidet filtrering...")
        ships_in_agder = filter_ships_in_agder(
            root_folder,
            date,
            radius,
            center_point,
            20  # Økt fra 10 til 20
        )

    print(f"Resultat av filtrering: {type(ships_in_agder)}")
    if ships_in_agder is not None:
        print(f"Antall skip funnet: {0 if ships_in_agder.empty else len(ships_in_agder)}")

    # Vis skipene på tabellform og visualiser
    if ships_in_agder is not None and not ships_in_agder.empty:
        # Velg kolonner å vise
        columns = ['mmsi', 'ship_name', 'ship_type', 'longitude', 'latitude']

        # Velg bare kolonner som faktisk finnes i datasettet
        available_columns = [kol for kol in columns if kol in ships_in_agder.columns]
        print(f"Tilgjengelige kolonner: {available_columns}")

        styled_tabell = ships_in_agder[available_columns].style\
            .format({'longitude': '{:.4f}', 'latitude': '{:.4f}'})\
            .set_caption(f'Skip innenfor {radius} meter fra Kristiansand, dato: {date_str}')\
            .set_table_styles([
                {'selector': 'th', 'props': [('background-color', '#f2f2f2'),
                                          ('color', '#333'),
                                          ('font-weight', 'bold')]},
                {'selector': 'td', 'props': [('padding', '5px')]}
            ])

        #display(styled_tabell)

        # Tøm tidligere output
        status_output.clear_output()
        # Visualiser skipene
        map_of_ships = visualize_ships(ships_in_agder, define_agder_polygon())

        map_point = visualize_ships_near_point(ships_in_agder, center_point, radius)

        # Vis kartet
        print("Viser kart over nyeste skipsbevegelser ...")
        time.sleep(2)
        display(map_point)

    else:
        print(f"Ingen skip funnet innen {radius} meter fra Kristiansand for {date_str}.")
        print("Prøv å:")
        print("1. Øke radius for å dekke et større område")
        print("2. Sjekke at datafilene eksisterer i 'data/processed'-mappen")

class GeoDataHandler(FileSystemEventHandler):
    def __init__(self, incoming_dir, raw_dir, processed_dir, source_dir):
        self.incoming_dir = incoming_dir
        self.raw_dir = raw_dir
        self.processed_dir = processed_dir
        self.source_dir = source_dir

    def on_created(self, event):
        # Vi er kun interessert i filhendelser (ikke mappeopprettelser)
        if not event.is_directory:
            filepath = event.src_path
            filename = os.path.basename(filepath)

            # Sjekk om det er en parquet-fil
            if filename.endswith('.parquet'):
                print(f"Oppdaget ny fil: {filename}")

                # Vent litt for å sikre at filen er ferdig skrevet
                time.sleep(1)

                # Flytt filen til raw-mappen
                destination = os.path.join(self.raw_dir, filename)
                shutil.move(filepath, destination)

                # Prosesser filen
                self.process_file(destination)
                # Kjør visualisering

    def process_file(self, filepath):
        try:
            # Definer målsti for den konverterte filen
            base_filename = os.path.splitext(os.path.basename(filepath))[0]
            output_path = os.path.join(self.processed_dir, f"{base_filename}")

            # Konverter parquet til GeoParquet med time-partisjonering
            print(f"Konverterer og partisjonerer fil...")
            results = convert_parquet_to_geoparquet(
                filepath,
                output_path,
                partition_columns=["date_time_utc"]  # Vil bruke time-partisjonering
            )

            if results:
                print(f"Konvertering og partisjonering vellykket.")

            else:
                print(f"Konvertering feilet for {filepath}")

        except Exception as e:
            print(f"Feil ved prosessering av {filepath}: {str(e)}")
            # Logge feilen for senere analyse
            with open(os.path.join(os.path.dirname(self.processed_dir), "error.log"), "a") as log_file:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                log_file.write(f"{timestamp} - {filepath}: {str(e)}\n")

def start_monitoring(data_folder="./data"):
    global _current_observer, global_paths

    # Stopp eksisterende observer hvis den finnes
    if _current_observer is not None:
        _current_observer.stop()
        _current_observer.join()
        print("Stoppet eksisterende overvåking.")

    # Gjenbruk eksisterende mappestruktur eller opprett en ny
    if global_paths is None:
        global_paths = create_folders(data_folder)

    # Opprett event handler med den NYE GeoDataHandler-klassen
    event_handler = GeoDataHandler(
        global_paths["incoming_folder"],
        global_paths["raw_folder"],
        global_paths["processed_folder"],
        global_paths["source_folder"]
    )

    # Start observer
    _current_observer = Observer()
    _current_observer.schedule(event_handler, global_paths["incoming_folder"], recursive=False)
    _current_observer.start()

    print(f"Starter overvåking av mappen: {global_paths['incoming_folder']} ... \n")
    time.sleep(2)

    return _current_observer

def start_notebook_monitoring():
    observer = start_monitoring(data_dir)
    time.sleep(1)
    return observer

def stop_monitoring():
    global _current_observer
    if _current_observer is not None:
        _current_observer.stop()
        _current_observer.join()
        _current_observer = None
    else:
        print("Ingen observatør kjører")

def stop_notebook_monitoring():
    global notebook_observer
    if notebook_observer is not None:
        stop_monitoring()
        notebook_observer = None
        print("Overvåking stoppet.")
    else:
        print("Ingen aktiv overvåking å stoppe.")

def simulate_streaming(data_folder="./data", num_files=5, min_interval=2, max_interval=10):
    global global_paths

    # Gjenbruk eksisterende mappestruktur eller opprett en ny
    if global_paths is None:
        global_paths = create_folders(data_folder)

    source_dir = global_paths["source_folder"]
    incoming_dir = global_paths["incoming_folder"]

    # Finn alle parquet-filer i kildemappen
    files = glob.glob(os.path.join(source_dir, "*.parquet"))

    if not files:
        print(f"Ingen .parquet-filer funnet i {source_dir}")
        run_vis()
        return

    print(f"Fant {len(files)} nye filer. Starter simulert strømming... \n")

    i = 0
    for file in files:
        # Generer et tilfeldig tidsintervall
        wait_time = random.uniform(min_interval, max_interval)
        time.sleep(wait_time)

        filename = f"{i+1}_{os.path.basename(file)}"  # Legg til indeks for å unngå duplikater
        destination = os.path.join(incoming_dir, filename)

        # Kopier filen (ikke flytt, så vi kan bruke den flere ganger)
        shutil.move(file, destination)
        time.sleep(3)
        i = i + 1

        run_vis()

def start_notebook_simulation():

    # Sjekk om source-mappen inneholder filer
    source_files = glob.glob(os.path.join(global_paths['source_folder'], "*.parquet"))

    simulate_streaming(data_dir, num_files)
    print("Simulering fullført.")

def create_radius_slider(initial_value=10000):
    slider = widgets.IntSlider(
        value=initial_value,
        min=1000,
        max=50000,
        step=1000,
        description='Radius (m):',
        tooltip='Radius i meter fra senterpunkt',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )

    label = widgets.Label(value=f'{initial_value} meter')

    def update_label(change):
        label.value = f'{change["new"]} meter'

    slider.observe(update_label, names='value')

    return widgets.HBox([slider, label]), slider

def create_stream_button():
    button = widgets.Button(
        description='Start strømming',
        button_style='primary',
        tooltip='Klikk for å starte strømming av data'
    )

    def on_click_handler(e):
        global notebook_observer
        status_output.clear_output()
        with status_output:
            notebook_observer = start_notebook_monitoring()
            start_notebook_simulation()

    button.on_click(on_click_handler)
    return button

def setup_ui():
    global radius_slider

    radius_widget, radius_slider = create_radius_slider()
    stream_button = create_stream_button()

    display(radius_widget)
    display(stream_button)
    display(status_output)

# Strømming

In [8]:
# Kall funksjonen for å sette opp hele UI-et
setup_ui()

Button(button_style='primary', description='Start strømming', style=ButtonStyle(), tooltip='Klikk for å starte…

Output()

## Stopp overvåking.

In [10]:
stop_notebook_monitoring()

Overvåking stoppet.
